In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict
matches = pd.read_csv('atp_matches_with_surface_elo.csv')

In [6]:
def create_player_elo_dataset(matches_df):
    """
    Cria um dataset com jogadores e seus ELOs por superfície
    
    Args:
        matches_df: DataFrame com todas as partidas (deve conter colunas:
                   winner_name, loser_name, surface, winner_surface_elo, loser_surface_elo)
    
    Returns:
        DataFrame com jogadores e ELOs por superfície
    """
    # Dicionário para armazenar os dados
    players_data = defaultdict(dict)
    
    # Lista única de todas as superfícies presentes nos dados
    surfaces = matches_df['surface'].dropna().unique()
    
    # Processar cada partida para coletar ELOs
    for _, row in matches_df.iterrows():
        if pd.isna(row['surface']):
            continue
            
        # Processar vencedor
        winner = row['winner_name']
        if winner not in players_data:
            players_data[winner] = {'name': winner}
        players_data[winner][f'elo_{row["surface"]}'] = row['winner_surface_elo']
        
        # Processar perdedor
        loser = row['loser_name']
        if loser not in players_data:
            players_data[loser] = {'name': loser}
        players_data[loser][f'elo_{row["surface"]}'] = row['loser_surface_elo']
    
    # Converter para DataFrame
    players_df = pd.DataFrame(players_data.values())
    
    # Adicionar estatísticas resumidas
    for surface in surfaces:
        if f'elo_{surface}' in players_df.columns:
            players_df[f'matches_{surface}'] = players_df.apply(
                lambda x: count_matches(matches_df, x['name'], surface), axis=1)
    
    # Calcular ELO médio (ponderado por número de partidas em cada superfície)
    players_df['elo_overall'] = players_df.apply(calculate_weighted_elo, axis=1, surfaces=surfaces)
    
    # Ordenar por ELO geral
    players_df = players_df.sort_values('elo_overall', ascending=False)
    
    return players_df.reset_index(drop=True)

def count_matches(matches_df, player_name, surface):
    """Conta quantas partidas um jogador teve em determinada superfície"""
    return len(matches_df[
        ((matches_df['winner_name'] == player_name) | 
         (matches_df['loser_name'] == player_name)) & 
        (matches_df['surface'] == surface)
    ])

def calculate_weighted_elo(row, surfaces):
    """Calcula ELO médio ponderado pelo número de partidas em cada superfície"""
    total_matches = 0
    weighted_sum = 0
    
    for surface in surfaces:
        elo_col = f'elo_{surface}'
        matches_col = f'matches_{surface}'
        
        if elo_col in row and matches_col in row and not pd.isna(row[elo_col]):
            matches = row[matches_col]
            if matches > 0:
                weighted_sum += row[elo_col] * matches
                total_matches += matches
    
    return weighted_sum / total_matches if total_matches > 0 else np.nan

In [7]:
players_elo = create_player_elo_dataset(matches)

In [8]:
# 3. Salve o resultado
players_elo.to_csv('players_elo_by_surface.csv', index=False)

# 4. Visualize
print(players_elo.head())

                    name     elo_Hard   elo_Carpet     elo_Clay    elo_Grass  \
0          Jannik Sinner  2213.912883          NaN  1832.195931  1721.237624   
1         Novak Djokovic  2108.466194  1603.140946  2026.777276  2013.418349   
2          Roger Federer  2053.490748  1787.780354  1940.660892  1949.636711   
3  Juan Martin del Potro  2018.416677  1554.121507  1857.979118  1751.312286   
4         Carlos Alcaraz  1961.072130          NaN  1971.619828  1787.920476   

   matches_Hard  matches_Carpet  matches_Clay  matches_Grass  elo_overall  
0           245               0            78             30  2087.697075  
1           848              13           361            141  2072.178099  
2           953              69           300            223  2004.725425  
3           433               8           122             58  1955.969337  
4           139               0            99             27  1947.370724  


In [10]:
# Carregar o dataset original
players_elo = pd.read_csv('players_elo_by_surface.csv')

# Lista de colunas para remover relacionadas a carpete
columns_to_drop = [col for col in players_elo.columns 
                  if 'carpet' in col.lower()]

# Mostrar quais colunas serão removidas
print("Colunas a serem removidas:", columns_to_drop)

# Remover as colunas (se existirem)
if columns_to_drop:
    players_elo = players_elo.drop(columns=columns_to_drop)
    print("\nColunas de carpete removidas com sucesso!")
else:
    print("\nNenhuma coluna de carpete encontrada no dataset.")

# Salvar o novo dataset
players_elo.to_csv('players_elo_by_surface_filtered.csv', index=False)

# Mostrar as colunas restantes
print("\nColunas no dataset filtrado:")
print(players_elo.columns.tolist())

Colunas a serem removidas: ['elo_Carpet', 'matches_Carpet']

Colunas de carpete removidas com sucesso!

Colunas no dataset filtrado:
['name', 'elo_Hard', 'elo_Clay', 'elo_Grass', 'matches_Hard', 'matches_Clay', 'matches_Grass', 'elo_overall']
